In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

import datajoint as dj
import tensorflow as tf
import matplotlib.pyplot as plt
import numpy as np

import seaborn as sns
sns.set_style('ticks', rc={'image.cmap': 'bwr'})

import os
import sys
import inspect

p = !pwd
p = os.path.dirname(os.path.dirname(p[0]))
if p not in sys.path:
    sys.path.append(p)

/usr/local/lib/python3.5/dist-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
from cnn_sys_ident.mesonet.data import MultiDataset
from cnn_sys_ident.mesonet.parameters import Core, Readout, Model, RegPath, Fit
from cnn_sys_ident.mesonet import MODELS

Connecting aecker@52.202.15.166:3306


In [3]:
data_key = {'data_hash': 'cfcd208495d565ef66e7dff9f98764da'}
dataset = MultiDataset() & data_key

# Load a model

In [ ]:
num_filters = 16
model_rel = MODELS['HermiteSparse'] * dataset \
    & 'positive_feature_weights=False AND shared_biases=False' \
    & {'num_filters_2': num_filters}
key = (Fit() * model_rel).fetch(dj.key, order_by='val_loss', limit=2)[1]
num_rotations = (model_rel & key).fetch1('num_rotations')
model = Fit().load_model(key)

### Find approximate receptive field locations

In [ ]:
masks = model.base.evaluate(model.readout.masks)
k = 50
m = np.exp(k*masks) / np.sum(np.exp(k*masks), axis=(1,2), keepdims=True)

ny, nx = m.shape[1:]
x = [27, 49]
y = [5, 28]
plt.figure(figsize=(20, 10))
plt.imshow(m.max(axis=0), cmap='gray')
plt.plot([x[0], x[0]], [0, ny-1], 'w')
plt.plot([x[1], x[1]], [0, ny-1], 'w')
plt.plot([0, nx-1], [y[0], y[0]], 'w')
plt.plot([0, nx-1], [y[1], y[1]], 'w')
plt.colorbar()
plt.show()

## Generate set of Gabors

In [ ]:
from cnn_sys_ident.utils.stimuli import GaborSet

In [ ]:
canvas_size = (64, 36)
center_range = (27, 49, 5, 28)
sizes = 8 * 1.3 ** np.arange(8)
spatial_frequencies = 1 * 1.35 ** np.arange(-1, 3)
contrasts = 2.0 ** np.arange(-5, 1)
num_orientations = 12
num_phases = 8
g = GaborSet(canvas_size, center_range, sizes, spatial_frequencies,
             contrasts, num_orientations, num_phases)

In [ ]:
for idx in np.random.randint(np.prod(g.num_stims), size=(10,)):
    plt.imshow(g.gabor_from_idx(idx), vmin=-1, vmax=1)
    plt.colorbar()
    plt.show()    

# Database tables

In [6]:
from cnn_sys_ident.mesonet.insilico import GaborParams, OptimalGabor

In [ ]:
GaborParams()

In [ ]:
OptimalGabor().populate()

# Size-contrast experiment

In [ ]:
from cnn_sys_ident.mesonet.insilico import SizeContrastTuning, SizeContrastTuningParams

In [ ]:
g = SizeContrastTuningParams().gabor_set(
    SizeContrastTuningParams().fetch1(dj.key),
    [64, 36], [12, 26], 1/8, np.pi/4, 0
)

In [ ]:
fig, axes = plt.subplots(12, 12, figsize=(18, 12))
for ax, img in zip(axes.flatten(), g.images()):
    ax.matshow(img, cmap='gray', vmin=-1, vmax=1)

# Orthogonal plaids

In [4]:
from cnn_sys_ident.mesonet.insilico import OrthPlaidsContrast, OrthPlaidsContrastParams